In [31]:
import numpy as np
import cv2
import os

In [55]:
class VideoProcess:
    def __init__(self, 
                 video_path=r'H:\Other Program\Program\PersonalProfit\VideoProcess\SourceData\dark_moving.mp4', 
                 image_save_dir=r'H:\Other Program\Program\PersonalProfit\VideoProcess\ImageSaveDir',
                 video_frame_interval=25,
                 dark_threshold=40,
                 dark_percent_threshold=0.8,
                 camera_pic_save_path=r'H:\Other Program\Program\PersonalProfit\VideoProcess\CameraOpen'
                 ):
        # 视频文件位置
        self.video_path = video_path
        # 视频帧图像保存位置
        self.image_save_dir = image_save_dir
        # 视频帧数间隔
        self.video_frame_interval = video_frame_interval
        # 图片灰度值阈值，像素点灰度值低于这个阈值认为是暗点
        self.dark_threshold = dark_threshold
        # 暗点占比阈值，大于这个阈值认为该图片是暗的
        self.dark_percent_threshold  =dark_percent_threshold
        # 摄像头拍摄的照片保存的目录
        self.camera_pic_save_path = camera_pic_save_path
        # 截取的图片的路径列表
        self.pic_path = []
        # 当前遍历到的图片的索引
        self.pic_index_pos = 0
        
    # 按指定帧数间隔截取视频并保存到指定位置
    def video_frame_capture(self):
        vc = cv2.VideoCapture(self.video_path)
        count = 1
        if vc.isOpened():
            rval,frame = vc.read()
            cv2.imwrite(self.image_save_dir + r'\\part_%d'%count + '.jpg',frame)
        else:
            rval = False
        while rval:
            rval, frame = vc.read()
            if count%self.video_frame_interval == 0:
                cv2.imwrite(self.image_save_dir + r'\\part_%d'%count + '.jpg',frame)
            count += 1
            cv2.waitKey(1)
        vc.release()
    
    # 输入一场图片，以及判断图片是不是暗的那个像素点占比分类阈值，
    # 判断这个图片是不是暗的，如果是暗的返回True，否则返回False
    def check_image_if_dark(self, pic_path):
        # 转化成灰度图
        img = cv2.imread(pic_path, cv2.IMREAD_GRAYSCALE)
        total_pixels = img.shape[0]*img.shape[1]
        dark_pixel_count = 0
        for row in img:
            row_count = np.array(row)
            dark_pixel_count += len(row_count[row_count < self.dark_threshold])
        return dark_pixel_count/total_pixels > self.dark_percent_threshold
            
    
    # 读取指定目录，返回目录下所有以.jpg结尾的图片路径
    def load_pic_paths(self, dir_path):
        res_pic_paths = []
        for root,firs,files in os.walk(dir_path):
            for file in files:
                if 'jpg' in file.split('.'):
                    res_pic_paths.append(root + r'\\'+file)
        return res_pic_paths
    
    # 控制开启摄像头,并拍摄一张照片保存,作为开启灯光的代替
    def open_camera_and_shot(self, camera_pic_save_path=r'H:\Other Program\Program\PersonalProfit\VideoProcess\CameraOpen'):
        print('开启灯光！（这里用摄像头拍张照代替，图片存储位置在：%s）'%camera_pic_save_path)
        cap = cv2.VideoCapture(0)
        f,frame = cap.read()  # 拍张照
        cv2.imwrite(camera_pic_save_path + r'\\' + 'camera_opened_flag.jpg', frame)
        cap.release() # 关闭调用的摄像头
    
    # 输入第一张暗图的索引,将这张暗图作为背景，根据后续的图像帧判断是否存在动态实体
    # 存在动态实体返回1， 不存在返回0
    def check_if_moving_conent(self, ):
    
    # 灯光控制主程
    def video_process_engine(self):
        # 先截取视频图片
        self.video_frame_capture()
        # 获取截取的图片的路径列表
        self.pic_path = self.load_pic_paths(self.image_save_dir)
        # 持续读取图片
        while self.pic_index_pos < len(self.pic_path):
            # 如果不是暗图，则啥也不做，
            if not self.check_image_if_dark(pic_path[self.pic_index_pos]):
                continue
            # 否则截取到暗图
            
            pass
    
    

In [56]:
video_path = r'H:\Other Program\Program\PersonalProfit\VideoProcess\SourceData\light_moving.mp4'
image_save_dir = r'H:\Other Program\Program\PersonalProfit\VideoProcess\ImageSaveDir'
video_processor = VideoProcess(video_path=video_path, image_save_dir=image_save_dir)
video_processor.video_frame_capture()

In [44]:
pic_path = r'H:\Other Program\Program\PersonalProfit\VideoProcess\ImageSaveDir\part_25.jpg'
video_processor.check_image_if_dark(pic_path)

False

In [45]:
tst_paths = video_processor.load_pic_paths(r'H:\Other Program\Program\PersonalProfit\VideoProcess\ImageSaveDir')

In [58]:
video_processor.open_camera_and_shot()